In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils
from wiki.newphet import load_prophet_prop
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [7]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'
VERSION = 'v7ty/'
TVERSION ='v7t/'

In [8]:
mediandf = pd.read_feather('../data/median_for_120_60.f') 

In [9]:
mediandf.page_index = mediandf.page_index.astype(int)
mediandf = mediandf.sort_values(by='page_index').reset_index().drop('index', axis=1)
mediandf.info()
mediandf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145063 entries, 0 to 145062
Data columns (total 15 columns):
page_index                  145063 non-null int64
Page                        145063 non-null object
median_pred_60              145063 non-null int64
median_pred_120             145063 non-null int64
prevYear_median_pred_60     145063 non-null int64
prevYear_median_pred_120    145063 non-null int64
smape_60_to_0               144151 non-null float64
smape_120_to_60             144123 non-null float64
prevYear_smape_60_to_0      141348 non-null float64
prevYear_smape_120_to_60    139615 non-null float64
nans_start_to_120           145063 non-null int64
nans_120_to_60              145063 non-null int64
nans_60_to_0                145063 non-null int64
prevYear_nans_60_to_0       145063 non-null int64
prevYear_nans_120_to_60     145063 non-null int64
dtypes: float64(4), int64(10), object(1)
memory usage: 16.6+ MB


,page_index,Page,median_pred_60,median_pred_120,prevYear_median_pred_60,prevYear_median_pred_120,smape_60_to_0,smape_120_to_60,prevYear_smape_60_to_0,prevYear_smape_120_to_60,nans_start_to_120,nans_120_to_60,nans_60_to_0,prevYear_nans_60_to_0,prevYear_nans_120_to_60
0,0,2NE1_zh.wikipedia.org_all-access_spider,24,23,16,17,32.278432,25.959053,40.870388,32.116348,0,0,0,0,0
1,1,2PM_zh.wikipedia.org_all-access_spider,24,22,14,17,42.847863,34.415304,60.431899,32.054860,0,0,0,0,0
2,2,3C_zh.wikipedia.org_all-access_spider,3,3,4,6,64.305367,65.870407,47.163640,56.748585,0,0,0,0,0
3,3,4minute_zh.wikipedia.org_all-access_spider,15,19,14,14,32.250349,33.896551,35.401843,49.278494,0,0,0,0,0
4,4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,14,14,3,5,48.547274,38.718059,54.426324,57.220372,291,0,0,0,0


In [10]:
val_df = pd.read_feather(PROPHET_PATH+RESULTS_PATH+VERSION[:-1]+'df.f'); 
val_df.page_index = val_df.page_index.astype(int)
val_df = val_df.sort_values(by='page_index').reset_index().drop('index', axis=1)
val_df = val_df.drop(VERSION[:-1]+'_train', axis=1)
print(val_df.shape)
print(val_df.info())
print(val_df[VERSION[:-1]+'_val'].mean())
val_df.head()

(145063, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145063 entries, 0 to 145062
Data columns (total 2 columns):
page_index    145063 non-null int64
v7ty_val      0 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.2 MB
None
nan


,page_index,v7ty_val
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN


In [13]:
test_df = pd.read_feather(PROPHET_PATH+RESULTS_PATH+VERSION[:-1]+'df.f'); 
test_df.page_index = test_df.page_index.astype(int)
test_df = test_df.sort_values(by='page_index').reset_index().drop('index', axis=1)
test_df = test_df.drop(VERSION[:-1]+'_train', axis=1)
print(test_df.shape)
print(test_df.info())
print(test_df[VERSION[:-1]+'_val'].mean())
test_df.head()

(145063, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145063 entries, 0 to 145062
Data columns (total 2 columns):
page_index    145063 non-null int64
v7ty_val      0 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.2 MB
None
nan


,page_index,v7ty_val
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN


In [14]:
df = val_df.merge(test_df, on='page_index'); print(df.shape)
df = df.merge(mediandf, on='page_index'); print(df.shape)
df['val_gain'] = df.smape_120_to_60 - df[VERSION[:-1]+'_val']
df['test_gain'] = df.smape_60_to_0 - df[VERSION[:-1]+'t_val']; df.head()

(145063, 3)
(145063, 17)


,page_index,v7_val,v7t_val,Page,median_pred_60,median_pred_120,prevYear_median_pred_60,prevYear_median_pred_120,smape_60_to_0,smape_120_to_60,prevYear_smape_60_to_0,prevYear_smape_120_to_60,nans_start_to_120,nans_120_to_60,nans_60_to_0,prevYear_nans_60_to_0,prevYear_nans_120_to_60,val_gain,test_gain
0,0,37.740052,32.642601,2NE1_zh.wikipedia.org_all-access_spider,24,23,16,17,32.278432,25.959053,40.870388,32.116348,0,0,0,0,0,-11.780999,-0.364169
1,1,37.009109,83.565553,2PM_zh.wikipedia.org_all-access_spider,24,22,14,17,42.847863,34.415304,60.431899,32.054860,0,0,0,0,0,-2.593805,-40.717689
2,2,173.376068,120.451293,3C_zh.wikipedia.org_all-access_spider,3,3,4,6,64.305367,65.870407,47.163640,56.748585,0,0,0,0,0,-107.505661,-56.145926
3,3,66.670011,38.493515,4minute_zh.wikipedia.org_all-access_spider,15,19,14,14,32.250349,33.896551,35.401843,49.278494,0,0,0,0,0,-32.773461,-6.243166
4,4,194.154882,89.498846,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,14,14,3,5,48.547274,38.718059,54.426324,57.220372,291,0,0,0,0,-155.436824,-40.951572


In [22]:
df.smape_120_to_60.mean()

40.059152930509576

In [21]:
(df[df['val_gain']>0].v7_val.sum() + df[~(df['val_gain']>0)].smape_120_to_60.sum())/len(df)

38.406838668421678

# New Game - Find the serieses where Median did bad a year ago over that period
* then try using prophet for those instead
* or where prophet did better than it a year ago?

* DO
    * Find median SMAPE scores over period of 2017-05-04 - 2017-07-02
        * Make function which gets median SMAPE over 60 day period
    * Try choosing those medians that do worse than prophet? Unlikely to be good as prophet is training on that
    * Try choosing worst X?
    * Try choosing worst X with a limit for small numbers worries giving huge SMAPES

In [37]:
train = pd.read_csv('../data/train_2.csv').drop('Page', axis=1); train.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,...,37.0,58.0,25.0,23.0,25.0,47.0,44.0,26.0,24.0,21.0
1,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,...,14.0,34.0,16.0,31.0,34.0,33.0,32.0,29.0,34.0,25.0
2,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,...,6.0,23.0,11.0,8.0,6.0,13.0,12.0,8.0,12.0,9.0
3,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,...,21.0,42.0,23.0,17.0,11.0,21.0,20.0,20.0,110.0,15.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,41.0,25.0,22.0,20.0,24.0,29.0,20.0,17.0,24.0


In [ ]:
def get_smape_df():
    denominator = train + preds / 200
    diff = np.abs(train - preds) / denominator
    diff[denominator == 0] = 0.0
    #return np.nanmean(diff, axis=0)
    return diff

In [ ]:
def smape_period(preds, period):
    denominator = train2.iloc[:,period[0]:period[1]].add(preds, axis=0) / 200
    diff = np.abs(train2.iloc[:,period[0]:period[1]].subtract(preds, axis=0)) / denominator
    diff[denominator == 0] = 0.0
    #return np.nanmean(diff, axis=0)
    return diff.mean(axis=1, skipna=True)